<a href="https://colab.research.google.com/github/ParkEunHyeok/AI_Study/blob/main/NLP/Bert_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 케라스에서 bert모델을 활용하기 쉽게 해주도록 모듈 설치,
# RAdam은 Adam optimizer의 수정판
!pip install keras-bert
!pip install keras-radam

In [2]:
# gdrive와 colab 연동
import os
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# 해당 경로로 들어가지는 것을 보아 연동이 잘 된 것을 확인할 수 있음
os.listdir('gdrive/My Drive/Colab Notebooks/bert')
# 빠르게 불러오기 위해 미리 path로 지정
path = "gdrive/My Drive/Colab Notebooks/bert"

In [4]:
import warnings
warnings.filterwarnings(action="ignore")

In [5]:
%tensorflow_version 1.x 
import tensorflow as tf

import pandas as pd
import numpy as np
import re
import pickle
import urllib.request

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
from tqdm import tqdm
import shutil

TensorFlow 1.x selected.


Using TensorFlow backend.


In [6]:
# bert 모형 활용에 필요한 모듈들
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

from keras_radam import RAdam

In [7]:
# 폴더를 복사하는 함수를 정의하여
# gdrive 안에 있는 bert 모형을 Colab 클라우드에 저장하여 로딩 속도 단축

def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

In [9]:
os.makedirs("bert")
copytree(os.path.join(path), "bert")

In [10]:
# 테스트 데이터셋 불러오기
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")
train = pd.read_table('ratings_train.txt')
test = pd.read_table('ratings_test.txt')

In [11]:
train[:5]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [12]:
# Bert 훈련을 위한 사전 설정
# SEQ_LEN은 문장의 최대 길이, 넘으면 짤림
# BATCH_SIZE는 메모리 초과를 방지하기 위해 작게 16으로 잡음
# 총 훈련 EPOCHS는 2로
# Learning Rate는 1e-5로 정함

SEQ_LEN = 128
BATCH_SIZE = 16
EPOCHS = 2
LR = 1e-5

In [13]:
# Pretrained_path는 사전 학습 모형이 있는 경로

pretrained_path ="bert"
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

DATA_COLUMN = "document"
LABEL_COLUMN = "label"

In [14]:
# vocab.txt에 있는 단어에 인덱스를 추가해주는 딕셔너리
# 분석할 문장 -> 토큰화 -> 인덱스(숫자) -> 버트 신경망 인풋

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
  for line in reader:
    token = line.strip()
    if "_" in token:
      token = token.replace("_", "")
      token = "##" + token
    token_dict[token] = len(token_dict)

In [15]:
# 단어에 인덱싱을 부여하여 토큰화
# Bert의 토큰화는 단어를 분리하는 방식
# 단어의 첫 시작은 ##가 붙지 않지만, 단어에 포함되면서 단어의 시작이 아닌 부분에는 ##가 붙음.
# Tokenizer 클래스를 상속받는다면 완전 자모 분리가 되므로,
# 이를 방지하기 위해 함수 재정의

class inherit_Tokenizer(Tokenizer):
  def _tokenize(self, text):
        if not self._cased:
            text = text
            
            text = text.lower()
        spaced = ''
        for ch in text:
            if self._is_punctuation(ch) or self._is_cjk_character(ch):
                spaced += ' ' + ch + ' '
            elif self._is_space(ch):
                spaced += ' '
            elif ord(ch) == 0 or ord(ch) == 0xfffd or self._is_control(ch):
                continue
            else:
                spaced += ch
        tokens = []
        for word in spaced.strip().split():
            tokens += self._word_piece_tokenize(word)
        return tokens

In [16]:
tokenizer = inherit_Tokenizer(token_dict)

In [17]:
# Tokenizing Test
# WordPiece Tokenizing 기법
tokenizer.tokenize("케라스로 버트 해보기 테스트 문장.")

['[CLS]',
 '케',
 '##라',
 '##스로',
 '버',
 '##트',
 '해',
 '##보',
 '##기',
 '테',
 '##스트',
 '문',
 '##장',
 '.',
 '[SEP]']

In [18]:
# 네이버 영화 댓글 감성분석 데이터를 버트 모형의 입력에 맞게 변형해주는 함수
# 함수 내부에 tokenizer.encode 함수가 버트 모형을 토큰화
# 토큰화된 단어를 인덱스에 맞게 숫자로 바꿔주게 됨.

def convert_data(data_df):
    global tokenizer
    indices, targets = [], []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        targets.append(data_df[LABEL_COLUMN][i])
    items = list(zip(indices, targets))
    
    indices, targets = zip(*items)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)], np.array(targets)

def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x, data_y = convert_data(data_df)

    return data_x, data_y

In [19]:
train_x, train_y = load_data(train)
test_x, test_y = load_data(test)

100%|██████████| 50000/50000 [00:09<00:00, 5354.12it/s]


In [20]:
train_x

[array([[  101,  9519,  9074, ...,     0,     0,     0],
        [  101,   100,   119, ...,     0,     0,     0],
        [  101,  9004, 32537, ...,     0,     0,     0],
        ...,
        [  101,  9638, 14153, ...,     0,     0,     0],
        [  101,  9751, 97707, ...,     0,     0,     0],
        [  101, 48556, 42428, ...,     0,     0,     0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [21]:
def sentence_convert_data(data):
    global tokenizer
    indices = []
    for i in tqdm(range(len(data))):
        print(tokenizer.tokenize(data[i]))
        ids, segments = tokenizer.encode(data[i], max_len=SEQ_LEN)
        indices.append(ids)
        
    items = indices
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def sentence_load_data(sentences):#sentence는 List로 받는다
    data_x = sentence_convert_data(sentences)
    return data_x

In [22]:
sentence_load_data(["케라스로 버트 해보기 정말 재밌음", "케라스 쉬워 쉬워"])

100%|██████████| 2/2 [00:00<00:00, 453.73it/s]

['[CLS]', '케', '##라', '##스로', '버', '##트', '해', '##보', '##기', '정', '##말', '재', '##밌', '##음', '[SEP]']
['[CLS]', '케', '##라', '##스', '쉬', '##워', '쉬', '##워', '[SEP]']


[array([[  101,  9806, 17342, 94980,  9336, 15184,  9960, 30005, 12310,
          9670, 89523,  9659,   100, 32158,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,

In [27]:
layer_num = 12
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,
)

IndexError: ignored

In [ ]:
model.summary()